#### **PREPROCESADO COMPLETO: Limpieza de ruido, silencio, pre-énfasis**

In [1]:
import numpy as np
import librosa
import os
import librosa.display
from scipy.signal import butter, filtfilt
import librosa.display
import matplotlib.pyplot as plt
from IPython.display import Audio, display

try:
    import noisereduce as nr

    _HAS_NOISEREDUCE = True
except ImportError:
    _HAS_NOISEREDUCE = False

from utils_audio import preprocess_audio

/Users/angel/miniconda3/envs/pharmai/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Ruta al archivo de audio
audio_path = os.path.join(
    "data", "outputs", "6_audio", "efectos_adversos", "audio_9_angel.wav"
)
print(f"Procesando archivo: {audio_path}")

# Ruta a guardar las imágenes
img_path = os.path.join("..", "images", "audio", "preprocesado")

# Preprocesar el audio (solo devuelve y)
y = preprocess_audio(
    audio_path,
    sr=16000,
    trim_db=20.0,
    lowcut=300.0,
    highcut=3400.0,
    pre_coef=0.97,
    rms_target=0.1,
    reduce_noise_flag=True,
    plot=True,
    show=True,
    save_dir=img_path,
)

Procesando archivo: data/outputs/6_audio/efectos_adversos/audio_9_angel.wav


/Users/angel/Desktop/MASTER/SEGUNDO_CUATRI/AnalisisNoEstructurados/Trabajo/PharmAI/audio/utils_audio.py:271: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


[✔] Guardado waveform original en: ../images/audio/preprocesado/01_original_waveform.png


/Users/angel/Desktop/MASTER/SEGUNDO_CUATRI/AnalisisNoEstructurados/Trabajo/PharmAI/audio/utils_audio.py:290: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


[✔] Guardado spectrogram original en: ../images/audio/preprocesado/01_original_spectrogram.png
[✔] Guardado waveform denoised en: ../images/audio/preprocesado/02_denoised_waveform.png
[✔] Guardado spectrogram denoised en: ../images/audio/preprocesado/02_denoised_spectrogram.png


/Users/angel/Desktop/MASTER/SEGUNDO_CUATRI/AnalisisNoEstructurados/Trabajo/PharmAI/audio/utils_audio.py:271: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


[✔] Guardado waveform trim en: ../images/audio/preprocesado/03_trim_waveform.png


/Users/angel/Desktop/MASTER/SEGUNDO_CUATRI/AnalisisNoEstructurados/Trabajo/PharmAI/audio/utils_audio.py:290: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


[✔] Guardado spectrogram trim en: ../images/audio/preprocesado/03_trim_spectrogram.png
[✔] Guardado waveform pre-énfasis en: ../images/audio/preprocesado/04_preemphasis_waveform.png
[✔] Guardado spectrogram pre-énfasis en: ../images/audio/preprocesado/04_preemphasis_spectrogram.png
[✔] Guardado waveform bandpass en: ../images/audio/preprocesado/05_bandpass_waveform.png
[✔] Guardado spectrogram bandpass en: ../images/audio/preprocesado/05_bandpass_spectrogram.png
[✔] Guardado waveform RMS en: ../images/audio/preprocesado/06_rms_waveform.png
[✔] Guardado spectrogram RMS en: ../images/audio/preprocesado/06_rms_spectrogram.png

Todas las imágenes se han guardado en: ../images/audio/preprocesado

